In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import random
from tensorflow import keras


from tensorflow.keras import layers
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.utils import np_utils

from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score


from keras.layers import SimpleRNN, Embedding, Dense, LSTM, Dropout
from keras.models import Sequential






In [3]:
from google.colab import files
uploaded = files.upload()


Saving final_emotion.csv to final_emotion.csv


In [4]:
import io
inputs = pd.read_csv(io.StringIO(uploaded['final_emotion.csv'].decode('utf-8')))
inputs.head()
print(len(inputs))

1600


In [8]:
inputs = inputs.dropna()
print(len(inputs))
print(inputs)

1600
       Decision     anger      fear  ...   disgust  positive  negative
0      truthful  0.009524  0.009524  ...  0.000000  0.057143  0.028571
1      truthful  0.000000  0.000000  ...  0.000000  0.066667  0.000000
2      truthful  0.004831  0.004831  ...  0.004831  0.057971  0.019324
3      truthful  0.000000  0.007874  ...  0.007874  0.039370  0.015748
4      truthful  0.000000  0.000000  ...  0.000000  0.069444  0.000000
...         ...       ...       ...  ...       ...       ...       ...
1595  deceptive  0.007634  0.000000  ...  0.000000  0.045802  0.015267
1596  deceptive  0.008264  0.000000  ...  0.008264  0.049587  0.008264
1597  deceptive  0.007092  0.007092  ...  0.000000  0.070922  0.014184
1598  deceptive  0.011628  0.011628  ...  0.023256  0.034884  0.023256
1599  deceptive  0.015385  0.003846  ...  0.019231  0.050000  0.015385

[1600 rows x 11 columns]


In [5]:
df2 = inputs.reindex(np.random.permutation(inputs.index))
print(df2)

       Decision     anger      fear  ...   disgust  positive  negative
811    truthful  0.025000  0.016667  ...  0.016667  0.041667  0.041667
1458  deceptive  0.016529  0.016529  ...  0.016529  0.033058  0.033058
479   deceptive  0.000000  0.003268  ...  0.000000  0.058824  0.013072
754   deceptive  0.000000  0.000000  ...  0.000000  0.030675  0.006135
270    truthful  0.006667  0.000000  ...  0.000000  0.053333  0.013333
...         ...       ...       ...  ...       ...       ...       ...
1310  deceptive  0.005780  0.005780  ...  0.011561  0.046243  0.011561
732   deceptive  0.000000  0.000000  ...  0.000000  0.055046  0.004587
661   deceptive  0.000000  0.000000  ...  0.000000  0.078261  0.008696
937    truthful  0.009346  0.004673  ...  0.004673  0.032710  0.023364
71     truthful  0.009804  0.009804  ...  0.009804  0.107843  0.009804

[1600 rows x 11 columns]


In [6]:
labels = df2['Decision'].values.tolist()
print(labels)

['truthful', 'deceptive', 'deceptive', 'deceptive', 'truthful', 'truthful', 'truthful', 'truthful', 'deceptive', 'truthful', 'deceptive', 'truthful', 'truthful', 'truthful', 'truthful', 'deceptive', 'truthful', 'truthful', 'truthful', 'deceptive', 'truthful', 'deceptive', 'truthful', 'deceptive', 'deceptive', 'truthful', 'truthful', 'deceptive', 'deceptive', 'truthful', 'truthful', 'deceptive', 'truthful', 'truthful', 'truthful', 'deceptive', 'deceptive', 'deceptive', 'truthful', 'deceptive', 'deceptive', 'truthful', 'deceptive', 'truthful', 'deceptive', 'deceptive', 'truthful', 'truthful', 'deceptive', 'truthful', 'deceptive', 'truthful', 'truthful', 'deceptive', 'truthful', 'deceptive', 'truthful', 'deceptive', 'deceptive', 'truthful', 'truthful', 'truthful', 'deceptive', 'deceptive', 'truthful', 'deceptive', 'truthful', 'deceptive', 'truthful', 'truthful', 'truthful', 'truthful', 'truthful', 'deceptive', 'truthful', 'deceptive', 'truthful', 'deceptive', 'deceptive', 'truthful', 'tru

In [7]:
labels = np.array([0 if label == "deceptive" else 1 for label in labels])

In [8]:
print(labels)
print(len(labels))

[1 0 0 ... 0 1 1]
1600


In [9]:
del df2['Decision']
print(df2)

         anger      fear  anticipation  ...   disgust  positive  negative
811   0.025000  0.016667      0.033333  ...  0.016667  0.041667  0.041667
1458  0.016529  0.016529      0.016529  ...  0.016529  0.033058  0.033058
479   0.000000  0.003268      0.022876  ...  0.000000  0.058824  0.013072
754   0.000000  0.000000      0.012270  ...  0.000000  0.030675  0.006135
270   0.006667  0.000000      0.033333  ...  0.000000  0.053333  0.013333
...        ...       ...           ...  ...       ...       ...       ...
1310  0.005780  0.005780      0.017341  ...  0.011561  0.046243  0.011561
732   0.000000  0.000000      0.018349  ...  0.000000  0.055046  0.004587
661   0.000000  0.000000      0.034783  ...  0.000000  0.078261  0.008696
937   0.009346  0.004673      0.023364  ...  0.004673  0.032710  0.023364
71    0.009804  0.009804      0.009804  ...  0.009804  0.107843  0.009804

[1600 rows x 10 columns]


In [10]:
train_x = df2.to_numpy()
print(len(train_x))
a = train_x[0]
print(a)

1600
[0.025      0.01666667 0.03333333 0.025      0.01666667 0.025
 0.025      0.01666667 0.04166667 0.04166667]


In [11]:
split_frac1 = 0.8

idx1 = int(len(train_x) * split_frac1)
train_x, val_x = train_x[:idx1], train_x[idx1:]
train_y, val_y = labels[:idx1], labels[idx1:]

In [12]:
#print(train_x)
print(len(train_x))
print(len(train_y))
print(len(val_x))
print(len(val_y))


1280
1280
320
320


In [ ]:
clf = LogisticRegression()
clf.fit(train_x,train_y)
p = clf.predict(val_x)
print (accuracy_score(train_y,p))

Basic NN
 

In [13]:
model = tf.keras.models.Sequential()

model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(128,activation=tf.nn.relu))
model.add(tf.keras.layers.Dropout(rate=0.25))

model.add(tf.keras.layers.Dense(128,activation=tf.nn.relu))
model.add(tf.keras.layers.Dropout(rate=0.25))

model.add(tf.keras.layers.Dense(128,activation=tf.nn.relu))
model.add(tf.keras.layers.Dropout(rate=0.25))

model.add(tf.keras.layers.Dense(128,activation=tf.nn.relu))
model.add(tf.keras.layers.Dropout(rate=0.25))

model.add(tf.keras.layers.Dense(128,activation=tf.nn.relu))
model.add(tf.keras.layers.Dropout(rate=0.25))


model.add(tf.keras.layers.Dense(1,activation=tf.nn.sigmoid))


model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])

class TestCallback(tf.keras.callbacks.Callback):
    def __init__(self, test_data):
        self.test_data = test_data

    def on_epoch_end(self, epoch, logs={}):
        x, y = self.test_data
        loss, acc = self.model.evaluate(x, y, verbose=0)
        print('\nTesting loss: {}, acc: {}\n'.format(loss, acc))



model.fit(train_x, train_y, validation_data=(val_x, val_y),epochs= 50, callbacks=[TestCallback((val_x,val_y))])



    # model.fit(train_x,train_y,epochs=,batch_size=64,verbose=1)
    # _, accuracy = model.evaluate(val_x, val_y)
    # print('Accuracy: %.2f' % (accuracy*100))

Epoch 1/50
30/40 [=====================>........] - ETA: 0s - loss: 0.6942 - accuracy: 0.5000
Testing loss: 0.6927180886268616, acc: 0.5

40/40 [==============================] - 0s 8ms/step - loss: 0.6939 - accuracy: 0.5023 - val_loss: 0.6927 - val_accuracy: 0.5000
Epoch 2/50
39/40 [============================>.] - ETA: 0s - loss: 0.6936 - accuracy: 0.5064
Testing loss: 0.6914797425270081, acc: 0.534375011920929

40/40 [==============================] - 0s 4ms/step - loss: 0.6937 - accuracy: 0.5023 - val_loss: 0.6915 - val_accuracy: 0.5344
Epoch 3/50
36/40 [==========================>...] - ETA: 0s - loss: 0.6928 - accuracy: 0.5130
Testing loss: 0.6898488402366638, acc: 0.53125

40/40 [==============================] - 0s 4ms/step - loss: 0.6930 - accuracy: 0.5063 - val_loss: 0.6898 - val_accuracy: 0.5312
Epoch 4/50
40/40 [==============================] - ETA: 0s - loss: 0.6919 - accuracy: 0.5352
Testing loss: 0.6877990961074829, acc: 0.5625

40/40 [==============================] -

In [14]:

_, accuracy = model.evaluate(val_x, val_y)
print('Accuracy: %.2f' % (accuracy*100))

10/10 [==============================] - 0s 1ms/step - loss: 0.6768 - accuracy: 0.5500
Accuracy: 55.00
